insert header

# Title

## Business Understanding and Business Problem

### Cost of Errors

## Data Understanding

In [1]:
#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### Initial Data

In [2]:
#the four csv files provided
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
#probably do not need to link to imdb and tmdb *probably will not need
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [14]:
#will need to split genres
#will have to figure out what to do with multiple genres
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [20]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [19]:
movies_df.nunique()


movieId    9742
title      9737
genres      951
dtype: int64

In [15]:
len(movies_df)

9742

In [13]:
movies_df.genres.value_counts()

Drama                                                  1053
Comedy                                                  946
Comedy|Drama                                            435
Comedy|Romance                                          363
Drama|Romance                                           349
                                                       ... 
Adventure|Animation|Comedy|Fantasy|Romance|Sci-Fi         1
Animation|Fantasy|Sci-Fi|Thriller                         1
Adventure|Animation|Children|Comedy|Musical|Romance       1
Animation|Crime|Drama                                     1
Adventure|Fantasy|Musical|Romance                         1
Name: genres, Length: 951, dtype: int64

In [11]:
movies_df.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [5]:
# can probably drop 'timestamp'
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [22]:
ratings_df.rating.value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [12]:
tags_df.head(50)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [10]:
tags_df[tags_df['movieId'] == 2]

,userId,movieId,tag,timestamp
33,62,2,fantasy,1528843929
34,62,2,magic board game,1528843932
35,62,2,Robin Williams,1528843907
982,474,2,game,1137375552


## Exploratory Data Analysis

In [23]:
simple_ratings_df = ratings_df.drop('timestamp', axis=1)

In [24]:
simple_ratings_df.head(1)

,userId,movieId,rating
0,1,1,4.0


## Modeling

## Final Model

## Results/Conclusions

### Further Exploration/Questions